<a href="https://colab.research.google.com/github/Lupama2/IntroCUDA/blob/main/ICNPG_try_arrayfire_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#ArrayFire python

In [ ]:
# This is a simple example of using arrayfire and benchmarking one of its
# functions, "pinverse." See https://arrayfire.org for more functions.

# install arrayfire
!pip install arrayfire==3.8.0+cu112 -f https://repo.arrayfire.com/python/wheels/3.8.0/


In [ ]:
# test installation success
import arrayfire as af

af.set_backend('cuda')    # we're on a solid gpu
print(af.info_str())
print(af.randu(3,3))      # matrix of random numbers on gpu

In [ ]:
# run something simple on GPU
%%timeit
A = af.randu(1000, 20)    # create some data in a 1000x20 matrix
af.pinverse(A);           # psuedo inverse, modify to benchmark other functions
af.sync()                 # execute code

In [ ]:
# arrayfire has multiple device options
af.set_backend('cpu')     # switch to cpu

In [ ]:
# run something simple on CPU
%%timeit
A = af.randu(1000, 20)    # create some data in a 1000x20 matrix
af.pinverse(A);           # psuedo inverse, modify to benchmark other functions

# Modelo SIR en una red de poblaciones conectadas

Se le llama también modelo metapoblacional.

Vamos a simular un modelo para la evolución de la población susceptible $S$, infectada $I$ y recuperada $R$. Vamos a considerar $N$ poblaciones, descriptas por $S_i, I_i, R_i$ con $i=0,...,N-1$, cada una con la misma tasa de recuperación $\beta$ pero distinta tasa de transmissión $\beta_i$. Esta vez no van a ser independientes las poblaciones, sino que vamos a permitir que infectados viajen de una a otra población a un rate $A_{ij}$, con $A_{ii}=0$,

$$
\begin{align}
\frac{dS_i}{dt} &= -\beta_i S_i I_i \\
\frac{dI_i}{dt} &= \beta_i S_i I_i - \gamma I_i + \sum_j A_{ij} I_j - I_i \sum_j A_{ji} \\
\frac{dR_i}{dt} &= \gamma I_i
\end{align}
$$

Son $3N$ ecuaciones acopladas, 3 por cada población.

La resolución numérica más simple es por diferencias finitas, usando el esquema de Euler, con un paso de tiempo suficientemente chico $dt$. Si llamamos $S_i^n \equiv S_i(n dt)$, y lo mismo para $I_i^n$, y $R_i^n$, tenemos

$$
\begin{align}
S_i^{n+1} &= S_i^n - dt\; \beta_i S_i^{n} I_i^n \\
I_i^{n+1} &= I_i^n + dt\; \left[\beta_i S_i^n I_i^n - \gamma I_i^n
+ \sum_j A_{ij} I^n_j - I^n_i \sum_j A_{ji} \right]
\\
R_i^{n+1} &= R_i^n + dt\; \gamma I_i^n
\end{align}
$$

La idea básica es asignar a cada hilo el trabajo de avanzar un paso de Euler las tres variables de una población $i$. Antes necesitamos calcular el miembro derecho de las tres ecuaciones. Los hilos pueden calcular fácilmente los términos salvo los de conectividad que es un producto matrix-vector
del tipo $\sum_j A_{ij} I_j = {\bf A}.{\bf I}$ y $B_i = \sum_j A_{ji}$
o ${\bf B}={\bf A}^T.{\bf v}$ donde ${\bf v}$ es un vector con todos sus elementos igual a $1$.


In [ ]:
%%writefile sirCupyDense.py
#@title modelo SIR en red usando cupy dense

import cupy as cp

def simular(N,g,dt,k,filename):
  # Declarar y Alocar memoria para los arrays de device S, I, R y beta usando CuPy
  # ....
  S = cp.zeros(N, dtype=cp.float32)
  I = cp.zeros(N, dtype=cp.float32)
  R = cp.zeros(N, dtype=cp.float32)
  beta = cp.zeros(N, dtype=cp.float32)

  cp.random.seed(1234)
  A = cp.random.rand(N, N).astype(cp.float32)
  cp.fill_diagonal(A,cp.float32(0))
  A[A<0.7]=cp.float32(0) #matriz de conectividad rala

  # aqui vamos a guardar las derivadas con respecto al tiempo (arrays auxiliares)
  dSdt = cp.zeros(N, dtype=cp.float32)
  dIdt = cp.zeros(N, dtype=cp.float32)
  dRdt = cp.zeros(N, dtype=cp.float32)

  # Inicializar S[i]=0.999, I[i]=0.001, R[i]=0, y beta[i]=0.02+i*0.02 usando CuPy
  # ....
  S.fill(cp.float32(1.000))
  I.fill(cp.float32(0.000))
  R.fill(cp.float32(0.000))
  beta.fill(cp.float32(0.15))

  # lo necesitamos para el termino de conectividad negativa
  rA=cp.sum(A, axis=1)
  #alternativa
  #rA=cp.dot(A,cp.ones(N))


  I[0]=cp.float32(0.001)
  S[0]=cp.float32(0.999)

  print("S=",S, len(beta))
  print("I=",I, len(beta))
  print("R=",R, len(beta))
  print("beta=",beta, len(beta))

  ntot = 5000

  f = open(filename, "w")

  # loop de tiempo
  for p in range(ntot):
      # imprimir I[] en columnas

      cp.savetxt(f, I.reshape(1, -1), delimiter='\t', fmt='%f')

      # derivadas de S[],I[],R[]
      dSdt = -beta*S*I
      dIdt = -dSdt -g*I + k*cp.dot(A, I) - k*I*rA
      dRdt = g*I

      # paso de Euler
      S = S+dSdt*dt
      I = I+dIdt*dt
      R = R+dRdt*dt


N = 10 # nro de poblaciones
g = 0.1  # tasa de recuperacion
dt = 0.1  # paso de tiempo
k=0.001 #conectividad

simular(N,g,dt,k,"data_con.csv")
simular(N,g,dt,cp.float32(0.0),"data_sin.csv")


In [ ]:
!nvprof --openacc-profiling off python sirCupyDense.py

In [ ]:
#@title visualicemos los tiempos (ask chatgpt: "Plot 10 column csv file using Python")
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def dibujame(datafile):
  # Read in the data from the CSV file
  data1 = pd.read_csv(datafile, delimiter='\t', header=None)

  # Create an arange to use as the x-axis
  x1 = np.arange(len(data1))

  # Plot each column of data against the x-axis
  for col in data1.columns:
      plt.plot(x1, data1[col], label=col)

  # Add a legend and labels for the axes
  plt.legend()
  plt.xlabel('tiempo')
  plt.ylabel('Infectados')
  plt.title(datafile)

  # Show the plot
  plt.tight_layout()
  plt.show()

In [ ]:
dibujame("data_con.csv")
dibujame("data_sin.csv")

In [ ]:
%%writefile sirCupySparse.py

#@title modelo SIR en red usando cupy sparse

import cupy as cp
import cupyx
from cupyx.scipy.sparse import csr_matrix


def simular(N,g,dt,k,filename):
  # Declarar y Alocar memoria para los arrays de device S, I, R y beta usando CuPy
  # ....
  S = cp.zeros(N, dtype=cp.float32)
  I = cp.zeros(N, dtype=cp.float32)
  R = cp.zeros(N, dtype=cp.float32)
  beta = cp.zeros(N, dtype=cp.float32)

  cp.random.seed(1234)
  A = cp.random.rand(N, N).astype(cp.float32)
  cp.fill_diagonal(A,cp.float32(0))
  A[A<0.7]=cp.float32(0) #matriz de conectividad rala

  sA = cupyx.scipy.sparse.csr_matrix(A)
  print("non zero elements =",sA.nnz)
  print("conecctivity sparse =",sA)
  print("conecctivity dense =",A)

  # aqui vamos a guardar las derivadas con respecto al tiempo (arrays auxiliares)
  dSdt = cp.zeros(N, dtype=cp.float32)
  dIdt = cp.zeros(N, dtype=cp.float32)
  dRdt = cp.zeros(N, dtype=cp.float32)

  # Inicializar S[i]=0.999, I[i]=0.001, R[i]=0, y beta[i]=0.02+i*0.02 usando CuPy
  # ....
  S.fill(cp.float32(1.000))
  I.fill(cp.float32(0.000))
  R.fill(cp.float32(0.000))
  beta.fill(cp.float32(0.15))

  # lo necesitamos para el termino de conectividad negativa
  rA=cp.sum(A, axis=1)


  I[0]=cp.float32(0.001)
  S[0]=cp.float32(0.999)

  print("condiciones iniciales:")
  print("S=",S, len(beta))
  print("I=",I, len(beta))
  print("R=",R, len(beta))
  print("beta=",beta, len(beta))

  ntot = 5000

  f = open(filename, "w")

  # loop de tiempo
  for p in range(ntot):
      # imprimir I[] en columnas

      cp.savetxt(f, I.reshape(1, -1), delimiter='\t', fmt='%f')

      # derivadas de S[],I[],R[]
      dSdt = -beta*S*I
      dIdt = -dSdt -g*I + k*sA*I - k*I*rA
      dRdt = g*I

      # paso de Euler
      S = S+dSdt*dt
      I = I+dIdt*dt
      R = R+dRdt*dt


N = 10 # nro de poblaciones
g = 0.1  # tasa de recuperacion
dt = 0.1  # paso de tiempo
k=0.001 #conectividad

simular(N,g,dt,k,"data_con.csv")
#simular(N,g,dt,cp.float32(0.0),"data_sin.csv")


In [ ]:
!nvprof --openacc-profiling off python sirCupySparse.py

In [ ]:
!tail data_con.csv


In [ ]:
dibujame("data_con.csv")


In [ ]:
%%writefile sirAfSparse.py

#@title modelo SIR en red usando arrayfire sparse

import numpy as np
import arrayfire as af


def simular(N,g,dt,k,filename):
  # Declarar y Alocar memoria para los arrays de device S, I, R y beta usando CuPy
  # ....
  nS = np.zeros(N, dtype=np.float32)
  nI = np.zeros(N, dtype=np.float32)
  nR = np.zeros(N, dtype=np.float32)
  nbeta = np.zeros(N, dtype=np.float32)

  np.random.seed(1234)
  nA = np.random.rand(N, N).astype(np.float32)
  np.fill_diagonal(nA,np.float32(0))
  nA[nA<0.7]=np.float32(0) #matriz de conectividad rala

  # aqui vamos a guardar las derivadas con respecto al tiempo (arrays auxiliares)
  ndSdt = np.zeros(N, dtype=np.float32)
  ndIdt = np.zeros(N, dtype=np.float32)
  ndRdt = np.zeros(N, dtype=np.float32)

  # Inicializar S[i]=0.999, I[i]=0.001, R[i]=0, y beta[i]=0.02+i*0.02 usando CuPy
  # ....
  nS.fill(np.float32(1.000))
  nI.fill(np.float32(0.000))
  nR.fill(np.float32(0.000))
  nbeta.fill(np.float32(0.15))
  nI[0]=np.float32(0.001)
  nS[0]=np.float32(0.999)

  # lo necesitamos para el termino de conectividad negativa
  nrA=np.sum(nA, axis=1)
  #rsA=sA.sum(axis=1)

  S = af.from_ndarray(nS);
  I = af.from_ndarray(nI);
  R = af.from_ndarray(nR);
  dSdt = af.from_ndarray(ndSdt)
  dIdt = af.from_ndarray(ndIdt)
  dRdt = af.from_ndarray(ndRdt)
  beta = af.from_ndarray(nbeta)
  A = af.from_ndarray(nA)
  rA = af.from_ndarray(nrA)
  sA = af.sparse.create_sparse_from_dense(A)


  print("S=",S, len(nbeta))
  print("I=",I, len(nbeta))
  print("R=",R, len(nbeta))
  print("sA=",sA, len(nbeta))
  print("beta=",beta, len(beta))

  ntot = 5000

  f = open(filename, "w")

  # loop de tiempo
  for p in range(ntot):
      # imprimir I[] en columnas

      nI = I.to_ndarray()
      np.savetxt(f, nI.reshape(1, -1), delimiter='\t', fmt='%f')

      # derivadas de S[],I[],R[]
      dSdt = -beta*S*I
      #dIdt = -dSdt -g*I + k*sA*I - k*I*rA #este no lo entiendte como cupy
      dIdt = -dSdt -g*I + k*af.matmul(sA, I) - k*I*rA
      dRdt = g*I

      # paso de Euler
      S = S+dSdt*dt
      I = I+dIdt*dt
      R = R+dRdt*dt


N = 10 # nro de poblaciones
g = 0.1  # tasa de recuperacion
dt = 0.1  # paso de tiempo
k=0.001 #conectividad

simular(N,g,dt,k,"data_con_af.csv")


In [ ]:
!nvprof --openacc-profiling off python sirAfSparse.py

In [ ]:
dibujame("data_con_af.csv")


Como notaran, no da igual que antes con cupy. Eso se debe a que cupy y numpy no generan los mismos números al azar con la misma semilla

In [ ]:
# ojo: los generadores de numpy y cupy no generan los mismos numeros con la misma semilla
import numpy as np
import cupy as cp

np.random.seed(1234)
cp.random.seed(1234)

print(np.random.rand(2, 2).astype(np.float32))
print(cp.random.rand(2, 2).astype(cp.float32))

In [ ]:
#@title Solo si les da un error relacionado a "UTF..." corran esto...
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding